In [2]:
import cv2
import numpy as np
from datetime import datetime

# ====== 基本設定 ======
STU_ID = "學號: 611406012"
VIDEO_PATH = "road.mp4"     # 輸入影片
OUT_PATH = "StableLane.mp4"  # 輸出影片

# 輸出比例
scale = 0.5
smooth_factor = 0.2  # 平滑係數，越小越穩定但延遲越高
prev_left_fit, prev_right_fit = None, None
# ======================

def put_id_stamp(img):
    """右上角加學號與時間"""
    ts = datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    h, w = img.shape[:2]
    cv2.putText(img, STU_ID, (w - 310, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2, cv2.LINE_AA)
    cv2.putText(img, ts, (w - 310, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0,255,255), 2, cv2.LINE_AA)
    return img

def average_slope_intercept(lines):
    """平均化線條的斜率與截距"""
    slopes, intercepts = [], []
    for x1, y1, x2, y2 in lines:
        if x2 == x1: continue
        slope = (y2 - y1) / (x2 - x1)
        intercept = y1 - slope * x1
        slopes.append(slope)
        intercepts.append(intercept)
    if slopes:
        return np.mean(slopes), np.mean(intercepts)
    return None

def make_line_points(y1, y2, slope, intercept):
    """由斜率與截距取得線段座標"""
    if slope is None: return None
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    return (x1, int(y1)), (x2, int(y2))

# 讀取影片
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("❌ 無法開啟影片")
    exit()

ret, frame = cap.read()
if not ret:
    print("❌ 無法讀取影片")
    cap.release()
    exit()

orig_height, orig_width = frame.shape[:2]
fps = cap.get(cv2.CAP_PROP_FPS)
out_width, out_height = int(orig_width * scale), int(orig_height * scale)

# 建立輸出影片
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUT_PATH, fourcc, fps, (out_width, out_height))

print("🎥 開始偵測影片中... 按 Q 結束")

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
while True:
    ret, frame = cap.read()
    if not ret: break

    img = cv2.resize(frame, (out_width, out_height))
    height, width = img.shape[:2]

    # ====== Step 1: 前處理 ======
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(gray, kernel, iterations=1)
    blur = cv2.GaussianBlur(dilated, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 120)

    # ====== Step 2: ROI ======
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (int(0.25 * width), height),
        (int(0.35 * width), int(0.8 * height)),
        (int(0.6 * width), int(0.8 * height)),
        (int(0.7 * width), height)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    masked_edges = cv2.bitwise_and(edges, mask)

    # ====== Step 3: Hough Transform ======
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi/180, threshold=30,
                            minLineLength=40, maxLineGap=120)

    left_lines, right_lines = [], []
    if lines is not None:
        for x1, y1, x2, y2 in lines[:, 0]:
            if x2 == x1: continue
            slope = (y2 - y1) / (x2 - x1)
            if slope < -0.3:
                left_lines.append((x1, y1, x2, y2))
            elif slope > 0.3:
                right_lines.append((x1, y1, x2, y2))

    left_fit = average_slope_intercept(left_lines)
    right_fit = average_slope_intercept(right_lines)

    # ====== Step 4: 平滑化 ======
    global prev_left_fit, prev_right_fit
    if left_fit is not None:
        if prev_left_fit is not None:
            left_fit = (
                smooth_factor * left_fit[0] + (1 - smooth_factor) * prev_left_fit[0],
                smooth_factor * left_fit[1] + (1 - smooth_factor) * prev_left_fit[1],
            )
        prev_left_fit = left_fit
    else:
        left_fit = prev_left_fit

    if right_fit is not None:
        if prev_right_fit is not None:
            right_fit = (
                smooth_factor * right_fit[0] + (1 - smooth_factor) * prev_right_fit[0],
                smooth_factor * right_fit[1] + (1 - smooth_factor) * prev_right_fit[1],
            )
        prev_right_fit = right_fit
    else:
        right_fit = prev_right_fit

    # ====== Step 5: 畫出車道與中心 ======
    lane_img = img.copy()
    y_bottom = height
    y_top = int(0.8 * height)
    left_line = make_line_points(y_bottom, y_top, *left_fit) if left_fit else None
    right_line = make_line_points(y_bottom, y_top, *right_fit) if right_fit else None

    overlay = lane_img.copy()
    if left_line and right_line:
        pts = np.array([[left_line[0], left_line[1], right_line[1], right_line[0]]], np.int32)
        cv2.fillPoly(overlay, pts, (0, 255, 0))
        cv2.addWeighted(overlay, 0.4, lane_img, 0.6, 0, lane_img)

        mid_x = (left_line[0][0] + right_line[0][0]) // 2
        mid_y = left_line[0][1]
        cv2.circle(lane_img, (mid_x, mid_y), 6, (0, 0, 255), -1)

    # 水印與輸出
    lane_img = put_id_stamp(lane_img)
    out.write(lane_img)
    cv2.imshow("HW3 Lane Detection (611406012)", lane_img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ 影片已完成輸出：", OUT_PATH)



🎥 開始偵測影片中... 按 Q 結束
✅ 影片已完成輸出： StableLane.mp4
